In [1]:
import pickle
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from collections import Counter,defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.model_selection import train_test_split
style.use('fivethirtyeight')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from wordcloud import WordCloud
from scipy.cluster.hierarchy import dendrogram, linkage

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\craut\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
harv_cat_words = {k:set(v) for k,v in pickle.load(open("../data/harvard_bag.pickle","rb")).items()}
nrc_cat_words = pickle.load(open("../data/nrc_bag.pickle","rb"))
print(harv_cat_words.keys())
print(nrc_cat_words.keys())

dict_keys(['Positive', 'Negative', 'Time', 'Human', 'Object', 'Knowledge', 'Number', 'Communication', 'Place', 'Government'])
dict_keys(['trust', 'anger', 'fear', 'negative', 'sadness', 'surprise', 'positive', 'disgust', 'joy', 'anticipation'])


In [44]:
# nrc_cat_words = defaultdict(set)
# nrc_df = pd.read_csv("../data/nrc_tokenized_words.csv")
# nrc_df.head()
# categories = nrc_df.columns[2:]
# for i,r in nrc_df.iterrows():
#     w = r["word"]
#     if any(r[categories]) and not all(r[categories]):
#         for c in categories:
#             if r[c]:
#                 nrc_cat_words[c].add(w)
# print(len(nrc_cat_words))
# pickle.dump(nrc_cat_words,open("../data/nrc_bag.pickle","wb"))

10


In [96]:
BB = pd.read_csv("../data/Hot Stuff.csv")
BB.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15


In [97]:
low = lambda s: s.lower()
BB["song"] = BB["Song"].apply(low)
BB["artist"] = BB["Performer"].apply(low)
BB["timestep"] = pd.to_datetime(BB["WeekID"])
BB.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,song,artist,timestep
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11,don't you worry child,swedish house mafia featuring john martin,2012-12-15
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12,don't you worry child,swedish house mafia featuring john martin,2012-12-22
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13,don't you worry child,swedish house mafia featuring john martin,2012-12-29
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14,don't you worry child,swedish house mafia featuring john martin,2013-01-05
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15,don't you worry child,swedish house mafia featuring john martin,2013-01-12


In [30]:
df = pickle.load(open("df_concat.pickle","rb"))
df.head()

,song,artist,genius_raw,genius_bag,az_lyrics_raw,az_lyrics_bag,lyrics_az_raw,lyrics_az_bag,GvAZL,GvLAZ,AZLvLAZ,genius_counter,az_lyrics_counter,lyrics_az_counter,word_intersect,word_intersect_counter
0,sauce it up,lil uzi vert,"[Intro]\nSauce it up, ayy, sauce it up, ayy\nS...","[sauce, it, up, ayy, sauce, it, up, ayy, sauce...","Sauce it up, ayy, sauce it up, ayy<br>\nSauce ...","[sauce, it, up, ayy, sauce, it, up, ayy, sauce...",[Intro]<br />\nSauce it up (aye)<br />\nSauce ...,"[sauce, it, up, aye, sauce, it, up, aye, sauce...",0.981573,0.852666,0.864487,"{'sauce': 18, 'it': 32, 'up': 36, 'ayy': 7, 'g...","{'sauce': 18, 'it': 32, 'up': 36, 'ayy': 7, 'i...","{'sauce': 12, 'it': 28, 'up': 24, 'aye': 6, 'g...","[let, okay, say, on, harley, brrr, woo, cardi,...","{'let': 2.0, 'okay': 1.0, 'say': 2.0, 'on': 4...."
1,backroad song,granger smith,Barbed wire fence carving out a hillside\nCutt...,"[barb, wire, fence, carve, out, a, hillside, c...","Barbed wire fence carving out a hillside, cutt...","[barb, wire, fence, carve, out, a, hillside, c...","Barbed wire fence carving out a hillside, cutt...","[barb, wire, fence, carve, out, a, hillside, c...",0.996372,0.997579,0.998789,"{'barb': 1, 'wire': 1, 'fence': 1, 'carve': 1,...","{'barb': 1, 'wire': 1, 'fence': 1, 'carve': 1,...","{'barb': 1, 'wire': 1, 'fence': 1, 'carve': 1,...","[let, feel, we're, today, grey, on, song, hear...","{'let': 1.0, 'feel': 11.0, 'we're': 1.0, 'toda..."
2,feelin' satisfied,boston,[Verse 1]\nWell come on\nAll you people\nThe t...,"[well, come, on, all, you, people, the, time, ...",Well come on<br>\nAll you people<br>\nThe time...,"[well, come, on, all, you, people, the, time, ...",Well come on<br />\nAll you people<br />\nThe ...,"[well, come, on, all, you, people, the, time, ...",0.949709,0.946203,0.996477,"{'well': 2, 'come': 9, 'on': 9, 'all': 1, 'you...","{'well': 2, 'come': 10, 'on': 11, 'all': 1, 'y...","{'well': 2, 'come': 10, 'on': 11, 'all': 1, 'y...","[people, let, can, alright, satisfy, feel, giv...","{'people': 1.0, 'let': 8.0, 'can': 2.0, 'alrig..."
3,saturday morning confusion,bobby russell,"Here they come, warming up\nI hear the pitter ...","[here, they, come, warm, up, i, hear, the, pit...",None,None,"Here they come, warming up<br />\nI hear the p...","[here, they, come, warm, up, i, hear, the, pit...",0.000000,0.999033,0.000000,"{'here': 3, 'they': 3, 'come': 3, 'warm': 2, '...",{},"{'here': 3, 'they': 3, 'come': 3, 'warm': 2, '...","[let, feel, dog, today, how, hear, on, come, r...","{'let': 2.0, 'feel': 1.0, 'dog': 4.0, 'today':..."
4,country man,luke bryan,"[Verse 1]\nYou need hands, rough not soft\nTo ...","[you, need, hand, rough, not, soft, to, come, ...","You need hands, rough not soft<br>\nTo come an...","[you, need, hand, rough, not, soft, to, come, ...","You need hands, rough not soft<br />\nTo come ...","[you, need, hand, rough, not, soft, to, come, ...",0.971121,0.967205,0.991220,"{'you': 7, 'need': 1, 'hand': 2, 'rough': 1, '...","{'you': 7, 'need': 1, 'hand': 2, 'rough': 1, '...","{'you': 7, 'need': 1, 'hand': 2, 'rough': 1, '...","[let, gator, tenni, move, come, way, muscle, t...","{'let': 1.0, 'gator': 1.0, 'tenni': 1.0, 'move..."


In [33]:
for i,r in df.iterrows():
    s,a,b = r["song"],r["artist"],r["word_intersect_counter"]
    print(s,a,b)
    break

sauce it up lil uzi vert defaultdict(<class 'int'>, {'let': 2.0, 'okay': 1.0, 'say': 2.0, 'on': 4.333333333333333, 'harley': 1.0, 'brrr': 1.0, 'woo': 1.0, 'cardi': 1.0, 'greatest': 1.0, 'these': 2.0, 'big': 2.0, 'and': 3.0, 'bird': 1.0, 'swerve': 1.0, 'hangup': 1.0, 'nut': 8.0, 'night': 1.0, 'a': 7.0, 'that': 9.0, 'enough': 20.0, 'my': 5.333333333333333, 'yeah': 8.666666666666666, 'you': 24.0, 'one': 3.0, 'money': 1.0, 'hey': 10.0, 'uzi': 1.6666666666666667, 'important': 1.0, 'foreign': 1.0, 'nice': 1.0, 'stack': 1.0, 'know': 1.6666666666666667, 'enormou': 1.0, 'morn': 1.0, 'maury': 1.0, 'have': 2.6666666666666665, 'only': 1.0, 'carti': 2.0, 'cash': 1.0, 'playboi': 1.0, 'all': 4.0, 'wet': 1.0, 'diamond': 1.0, 'swear': 2.0, 'bad': 1.0, 'dumb': 9.0, 'ever': 1.0, 'phone': 1.0, 'ooh': 6.666666666666667, 'knot': 1.0, 'lamb': 1.0, 'think': 2.0, 'vert': 1.6666666666666667, 'me': 5.666666666666667, 'meet': 1.0, 'it': 30.666666666666668, 'nigga': 3.0, 'go': 18.666666666666668, 'boy': 1.0, 'much

In [70]:
def construct_song_cat_prop_table(cat_dict):
    columns = ["song","artist"]+list(cat_dict.keys())+["num_words"]
    cats = columns[2:-1]
    word_pool = {w for c in cat_dict for w in cat_dict[c]}
    print("lexicon has {} words".format(len(word_pool)))
    data = []
    for i,r in df.iterrows():
        s,a,b = r["song"],r["artist"],r["word_intersect_counter"]
        cnts = [0]*len(cats)
        props = [0.0]*len(cats)
        wc = 0
        for w in b:
            if w in word_pool:
                wc+=b[w]
                for i,cat in enumerate(cats):
                    if w in cat_dict[cat]:
                        cnts[i]+=b[w]
        props = [cnt/wc if wc > 0 else np.nan for cnt in cnts]
        dr = [s,a]+cnts+[wc]+props
        if wc > 0:
            data.append(dr)
    columns = ["song","artist"]+list(map(lambda c:"{}_cnts".format(c),cats))+["num_words"]+list(map(lambda c:"{}_prop".format(c),cats))
    return pd.DataFrame(data,columns=columns)

In [71]:
test_df = construct_song_cat_prop_table(harv_cat_words)
test_df.head()

lexicon has 698 words


,song,artist,Positive_cnts,Negative_cnts,Time_cnts,Human_cnts,Object_cnts,Knowledge_cnts,Number_cnts,Communication_cnts,...,Positive_prop,Negative_prop,Time_prop,Human_prop,Object_prop,Knowledge_prop,Number_prop,Communication_prop,Place_prop,Government_prop
0,sauce it up,lil uzi vert,2.0,10.0,1.0,5.0,2.0,0.0,13.0,0.0,...,0.060606,0.303030,0.030303,0.151515,0.060606,0.000000,0.393939,0.000000,0.000000,0.0
1,backroad song,granger smith,7.0,0.0,13.0,1.0,7.0,0.0,1.0,10.0,...,0.162791,0.000000,0.302326,0.023256,0.162791,0.000000,0.023256,0.232558,0.093023,0.0
2,feelin' satisfied,boston,0.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,...,0.000000,0.166667,0.333333,0.000000,0.000000,0.000000,0.166667,0.166667,0.166667,0.0
3,saturday morning confusion,bobby russell,2.0,9.0,20.0,12.5,7.0,1.0,5.0,1.0,...,0.033613,0.151261,0.336134,0.210084,0.117647,0.016807,0.084034,0.016807,0.033613,0.0
4,country man,luke bryan,1.0,1.0,1.0,5.0,4.0,0.0,4.0,0.0,...,0.055556,0.055556,0.055556,0.277778,0.222222,0.000000,0.222222,0.000000,0.111111,0.0


In [62]:
sum(test_df["num_words"]<3.0)

1345

In [125]:
Big_DF = BB.merge(test_df,left_on=["song","artist"],right_on=["song","artist"],how="inner")
print(len(Big_DF))
print(len(BB))
Big_DF.head()

271460
315295


,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,...,Positive_prop,Negative_prop,Time_prop,Human_prop,Object_prop,Knowledge_prop,Number_prop,Communication_prop,Place_prop,Government_prop
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11,...,0.337349,0.0,0.036145,0.301205,0.0,0.108434,0.0,0.072289,0.144578,0.0
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12,...,0.337349,0.0,0.036145,0.301205,0.0,0.108434,0.0,0.072289,0.144578,0.0
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13,...,0.337349,0.0,0.036145,0.301205,0.0,0.108434,0.0,0.072289,0.144578,0.0
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14,...,0.337349,0.0,0.036145,0.301205,0.0,0.108434,0.0,0.072289,0.144578,0.0
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15,...,0.337349,0.0,0.036145,0.301205,0.0,0.108434,0.0,0.072289,0.144578,0.0


In [129]:
Big_DF = Big_DF.sort_values(["timestep","Week Position"])
date_cnts = Big_DF.groupby("timestep").size().reset_index(name='song_counts')
date_cnts = date_cnts.set_index("timestep")
date_cnts.head()
#Big_DF.head()

,song_counts
timestep,
1958-08-02,72
1958-08-09,75
1958-08-16,66
1958-08-23,67
1958-08-30,61


In [103]:
Big_DF.columns[10:]

Index(['song', 'artist', 'timestep', 'Positive_cnts', 'Negative_cnts',
       'Time_cnts', 'Human_cnts', 'Object_cnts', 'Knowledge_cnts',
       'Number_cnts', 'Communication_cnts', 'Place_cnts', 'Government_cnts',
       'num_words', 'Positive_prop', 'Negative_prop', 'Time_prop',
       'Human_prop', 'Object_prop', 'Knowledge_prop', 'Number_prop',
       'Communication_prop', 'Place_prop', 'Government_prop'],
      dtype='object')

In [76]:
aggr_cols = ['Positive_cnts', 'Negative_cnts',
       'Time_cnts', 'Human_cnts', 'Object_cnts', 'Knowledge_cnts',
       'Number_cnts', 'Communication_cnts', 'Place_cnts', 'Government_cnts',
       'num_words']
avg_cols = ['Positive_prop', 'Negative_prop', 'Time_prop',
       'Human_prop', 'Object_prop', 'Knowledge_prop', 'Number_prop',
       'Communication_prop', 'Place_prop', 'Government_prop']

In [130]:
Big_DF = Big_DF.groupby("timestep").agg({i:np.mean for i in avg_cols}).join(Big_DF.groupby("timestep").agg({i:np.sum for i in aggr_cols}))
Big_DF = Big_DF.join(date_cnts)
Big_DF = Big_DF.reset_index()
Big_DF.head()

,timestep,Positive_prop,Negative_prop,Time_prop,Human_prop,Object_prop,Knowledge_prop,Number_prop,Communication_prop,Place_prop,...,Time_cnts,Human_cnts,Object_cnts,Knowledge_cnts,Number_cnts,Communication_cnts,Place_cnts,Government_cnts,num_words,song_counts
0,1958-08-02,0.200191,0.186458,0.121920,0.129282,0.110238,0.014093,0.142720,0.028587,0.065694,...,92.333333,107.833333,76.0,12.0,129.0,23.0,49.000000,1.0,783.000000,72
1,1958-08-09,0.224297,0.154150,0.112234,0.128485,0.118590,0.023358,0.140357,0.030797,0.067730,...,90.333333,105.833333,81.0,16.0,127.0,25.0,53.666667,0.0,801.666667,75
2,1958-08-16,0.197232,0.163229,0.129134,0.114400,0.132131,0.019378,0.136217,0.028130,0.076363,...,91.333333,89.833333,79.0,15.0,102.0,21.0,53.000000,3.0,715.000000,66
3,1958-08-23,0.213554,0.147857,0.120359,0.129381,0.121195,0.019088,0.134265,0.035457,0.078843,...,86.333333,103.833333,80.0,15.0,105.0,26.0,59.000000,0.0,734.000000,67
4,1958-08-30,0.214669,0.144477,0.120517,0.118993,0.109327,0.020966,0.152390,0.034683,0.083979,...,83.333333,95.500000,74.0,15.0,104.5,23.0,58.000000,0.0,698.166667,61


In [132]:
Big_DF.to_csv("../data/harvard_weekly_cat.csv")

In [131]:
print(len(Big_DF))

3153


In [134]:
test_df = construct_song_cat_prop_table(nrc_cat_words)
test_df.head()

lexicon has 5986 words


,song,artist,trust_cnts,anger_cnts,fear_cnts,negative_cnts,sadness_cnts,surprise_cnts,positive_cnts,disgust_cnts,...,trust_prop,anger_prop,fear_prop,negative_prop,sadness_prop,surprise_prop,positive_prop,disgust_prop,joy_prop,anticipation_prop
0,sauce it up,lil uzi vert,10.666667,14.333333,10.333333,32.666667,7.333333,5.000000,9.333333,12.333333,...,0.231884,0.311594,0.224638,0.710145,0.159420,0.108696,0.202899,0.268116,0.159420,0.152174
1,backroad song,granger smith,20.666667,6.000000,4.000000,8.000000,14.666667,1.000000,31.666667,3.000000,...,0.484375,0.140625,0.093750,0.187500,0.343750,0.023438,0.742188,0.070312,0.437500,0.320312
2,feelin' satisfied,boston,1.000000,1.000000,2.000000,2.000000,3.000000,3.666667,8.666667,2.000000,...,0.065217,0.065217,0.130435,0.130435,0.195652,0.239130,0.565217,0.130435,0.260870,0.260870
3,saturday morning confusion,bobby russell,12.000000,22.000000,18.000000,37.000000,14.000000,10.000000,23.000000,9.000000,...,0.181818,0.333333,0.272727,0.560606,0.212121,0.151515,0.348485,0.136364,0.151515,0.242424
4,country man,luke bryan,5.333333,4.000000,2.000000,8.000000,0.000000,3.000000,13.333333,4.000000,...,0.238806,0.179104,0.089552,0.358209,0.000000,0.134328,0.597015,0.179104,0.462687,0.238806


In [135]:
sum(test_df["num_words"]<3.0)

131

In [136]:
Big_DF = BB.merge(test_df,left_on=["song","artist"],right_on=["song","artist"],how="inner")
print(len(Big_DF))
print(len(BB))
Big_DF.head()

275194
315295


,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,...,trust_prop,anger_prop,fear_prop,negative_prop,sadness_prop,surprise_prop,positive_prop,disgust_prop,joy_prop,anticipation_prop
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11,...,0.416107,0.020134,0.456376,0.395973,0.355705,0.040268,0.442953,0.020134,0.275168,0.57047
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12,...,0.416107,0.020134,0.456376,0.395973,0.355705,0.040268,0.442953,0.020134,0.275168,0.57047
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13,...,0.416107,0.020134,0.456376,0.395973,0.355705,0.040268,0.442953,0.020134,0.275168,0.57047
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14,...,0.416107,0.020134,0.456376,0.395973,0.355705,0.040268,0.442953,0.020134,0.275168,0.57047
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15,...,0.416107,0.020134,0.456376,0.395973,0.355705,0.040268,0.442953,0.020134,0.275168,0.57047


In [137]:
Big_DF = Big_DF.sort_values(["timestep","Week Position"])
date_cnts = Big_DF.groupby("timestep").size().reset_index(name='song_counts')
date_cnts = date_cnts.set_index("timestep")
date_cnts.head()
#Big_DF.head()

,song_counts
timestep,
1958-08-02,75
1958-08-09,79
1958-08-16,70
1958-08-23,71
1958-08-30,65


In [138]:
Big_DF.columns[10:]

Index(['song', 'artist', 'timestep', 'trust_cnts', 'anger_cnts', 'fear_cnts',
       'negative_cnts', 'sadness_cnts', 'surprise_cnts', 'positive_cnts',
       'disgust_cnts', 'joy_cnts', 'anticipation_cnts', 'num_words',
       'trust_prop', 'anger_prop', 'fear_prop', 'negative_prop',
       'sadness_prop', 'surprise_prop', 'positive_prop', 'disgust_prop',
       'joy_prop', 'anticipation_prop'],
      dtype='object')

In [139]:
aggr_cols = ['trust_cnts', 'anger_cnts', 'fear_cnts',
       'negative_cnts', 'sadness_cnts', 'surprise_cnts', 'positive_cnts',
       'disgust_cnts', 'joy_cnts', 'anticipation_cnts', 'num_words']
avg_cols = ['trust_prop', 'anger_prop', 'fear_prop', 'negative_prop',
       'sadness_prop', 'surprise_prop', 'positive_prop', 'disgust_prop',
       'joy_prop', 'anticipation_prop']

In [140]:
Big_DF = Big_DF.groupby("timestep").agg({i:np.mean for i in avg_cols}).join(Big_DF.groupby("timestep").agg({i:np.sum for i in aggr_cols}))
Big_DF = Big_DF.join(date_cnts)
Big_DF = Big_DF.reset_index()
Big_DF.head()

,timestep,trust_prop,anger_prop,fear_prop,negative_prop,sadness_prop,surprise_prop,positive_prop,disgust_prop,joy_prop,...,fear_cnts,negative_cnts,sadness_cnts,surprise_cnts,positive_cnts,disgust_cnts,joy_cnts,anticipation_cnts,num_words,song_counts
0,1958-08-02,0.411349,0.118392,0.183048,0.325812,0.202441,0.143044,0.599196,0.100254,0.455321,...,319.666667,540.000000,332.000000,265.666667,1098.666667,160.833333,791.000000,459.666667,1797.833333,75
1,1958-08-09,0.401904,0.111387,0.175890,0.319436,0.206642,0.137407,0.592637,0.093282,0.433834,...,347.666667,587.666667,376.000000,277.000000,1140.333333,167.166667,805.666667,494.000000,1916.833333,79
2,1958-08-16,0.382215,0.103377,0.155576,0.290782,0.184506,0.139163,0.616364,0.074417,0.447688,...,286.666667,485.666667,308.000000,243.000000,1073.333333,121.166667,755.666667,468.000000,1718.833333,70
3,1958-08-23,0.387391,0.093500,0.154120,0.296855,0.188361,0.138984,0.617310,0.071227,0.443808,...,287.666667,509.166667,321.000000,249.000000,1095.333333,124.166667,760.666667,473.500000,1762.333333,71
4,1958-08-30,0.393765,0.092171,0.154056,0.281122,0.187542,0.137868,0.619837,0.061882,0.451476,...,268.833333,460.500000,298.666667,236.000000,1077.500000,111.833333,778.333333,450.833333,1701.833333,65


In [141]:
Big_DF.to_csv("../data/nrc_weekly_cat.csv")

In [142]:
print(len(Big_DF))

3153
